In [ ]:
from ursina import *
from ursina.prefabs.first_person_controller import FirstPersonController
import time

app = Ursina()

# Set the window background to black
window.clear_color = color.black

# Create a black sky
Sky(color=color.black)

# Flag to indicate when to show the story and disable player movement
show_story = True

# Create the player (First Person Controller) positioned to face the display screen
player = FirstPersonController(y=2, origin_y=-.5)  # Set y=2 so player starts close to ground level
player.position = (0, 0, 8)  # Position player in front of the display screen
player.rotation_y = 180  # Rotate player to face the display screen
player.enabled = False  # Disable player movement until the story ends

# Create a ground with a grid
ground = Entity(
    model=Grid(width=100, height=100),  # Larger grid for the floor
    scale=(100, 1, 100),  # Make the ground wide
    color=color.black,
    texture='white_cube',  # Using a simple texture for now
    texture_scale=(100, 100),
    collider='box'
)

# Create a display screen with a custom texture and text
def create_display_screen():
    display_box = Entity(
        model="quad",  # Use quad for a flat surface
        scale=(5, 3),  # Size of the display
        position=(0, 2, -5),  # Position in front of the player
        rotation=(0, 0, 0),  # Initial rotation
        collider='box',  # Collider for interaction
        texture='bigbrain.png'  # Apply texture directly to the display box
    )
    
    # Create text on the display with the updated message
    display_text = Text(
        text="Welcome to your brain, jump in your fear",  # Updated message
        parent=display_box,  # Attach to display box
        position=(0, 0, 0.06),  # Slightly in front of the box
        origin=(0, 0),  # Center the text on the box
        color=color.white,  # White text for visibility
        scale=1.5,  # Scale for better readability
        background=True  # Adds contrast against the background
    )

    # Create three red objects as children of the display box
    red_objects = []
    for i in range(3):
        red_object = Entity(
            model='cube',  # Model for red object
            color=color.red,  # Red color to simulate danger or emphasis
            scale=(0.5, 0.5, 0.1),  # Small size for each red object
            position=(-1 + i, -1, 0.06),  # Position them side-by-side
            parent=display_box  # Attach them to the display box
        )
        red_objects.append(red_object)  # Add each to a list if you need to reference them later

    return display_box

display_screen = create_display_screen()

# Initial story text
story_text_content = "You find yourself in a strange place, a landscape of your own mind.\nWelcome to your thoughts and fears.\nAre you ready to face them?"

# Story dialogue overlay, initially blank for typing animation
story_text = Text(
    text='',
    scale=0.8,  # Smaller text
    origin=(0, 0),
    background=True,
    enabled=True  # Start with the text visible
)

# Typing effect variables
typing_speed = 0.05  # Time delay between characters
current_text = ""  # Initialize as empty string for typing effect
typing_index = 0
story_duration = 10  # Total display duration for the story text
story_start_time = time.time()

# Flag for triggering the fall event
falling = False
fall_timer = 0
fall_duration = 2  # Time before falling into the red room

# Red room (a room with four red walls)
def create_red_room():
    room_size = 20
    walls = []

    # Create four walls for the red room
    for i in range(4):
        wall = Entity(
            model='cube',
            color=color.red,
            scale=(room_size, 10, 1),
            position=(0, 5, 0),
            collider='box'
        )
        if i == 0:  # Front wall
            wall.position = (0, 5, -room_size / 2)
        elif i == 1:  # Back wall
            wall.position = (0, 5, room_size / 2)
        elif i == 2:  # Left wall
            wall.rotation_y = 90
            wall.position = (-room_size / 2, 5, 0)
        elif i == 3:  # Right wall
            wall.rotation_y = 90
            wall.position = (room_size / 2, 5, 0)
        walls.append(wall)

    return walls

red_room_walls = None

# Update function for typing animation, gravity, and proximity to the box
def update():
    global typing_index, show_story, current_text, falling, fall_timer, red_room_walls  # Declare falling and fall_timer as global

    # Check if story time has elapsed
    if show_story:
        # Typing animation: add one character at a time based on typing speed
        if typing_index < len(story_text_content):
            current_text += story_text_content[typing_index]
            story_text.text = current_text  # Update displayed text
            typing_index += 1
            time.sleep(typing_speed)  # Delay between characters
        else:
            # When typing is done, check for elapsed time
            if time.time() - story_start_time > story_duration:
                show_story = False
                story_text.enabled = False  # Hide story text
                player.enabled = True  # Enable player movement and rotation
    
    # Check if player is near the display box
    if player.position.distance_to(display_screen.position) < 3 and not falling:
        falling = True  # Start the fall process

    # If falling, apply gravity and start the countdown
    if falling:
        fall_timer += time.dt
        player.y -= time.dt * 2  # Apply fall speed
        if fall_timer > fall_duration:
            if red_room_walls is None:  # Only create the red room once
                red_room_walls = create_red_room()
            player.position = (0, 5, 0)  # Position player in the red room
            player.y = 5  # Make sure player stays within the red room
            falling = False  # Stop the fall
            
            from ursina import *
from ursina.prefabs.first_person_controller import FirstPersonController
import random

# Initialize the app
app = Ursina()

# Set game icon (replace with your own icon file)
window.icon = 'PNG.webp'

# Variables
floor_level = 0
fear_meter = 0
power_up_entities = []
shadow_entities = []
player_speed_boost = 1  # Normal speed
ambient_sound = Audio('assets/fear_sound.mp3', loop=True, autoplay=True)  # Add your eerie sound file here

# Create Burj Khalifa structure
def create_burj_khalifa(position=(0, 0, 0)):
    heights = [2, 1.5, 1.2, 1, 0.8, 0.5, 0.2]
    current_height = 0
    for i, height in enumerate(heights):
        Entity(
            model='cylinder',
            position=(position[0], position[1] + current_height + height / 2, position[2]),
            scale=(1 - i * 0.1, height, 1 - i * 0.1),
            color=color.gray,
            texture='white_cube',
            texture_scale=(1, height)
        )
        current_height += height

    Entity(
        model='cone',
        position=(position[0], position[1] + current_height + 0.5, position[2]),
        scale=(0.2, 1, 0.2),
        color=color.gray,
        texture='white_cube'
    )

# Initialize shadow entities
def initialize_shadows():
    global shadow_entities
    for i in range(5):
        shadow = Entity(
            model='sphere',
            position=(i * 3, floor_level + 0.5, i * 3),
            scale=0.5,
            color=color.black,
            texture='assets/shadow.png'
        )
        shadow_entities.append(shadow)

# Move shadows towards the player
def move_shadows():
    for shadow in shadow_entities:
        if (shadow.position - player.position).length() < 10:
            shadow.position += (player.position - shadow.position).normalized() * time.dt * 2

# Spawn power-ups
def spawn_power_ups(count=3):
    global power_up_entities
    power_up_entities = []
    for _ in range(count):
        power_up = Entity(
            model='sphere',
            position=(random.uniform(-10, 10), floor_level + 0.5, random.uniform(-10, 10)),
            scale=0.5,
            color=color.blue,
            texture='white_cube',
        )
        power_up_entities.append(power_up)

# Collect power-ups
def collect_power_ups():
    global player_speed_boost
    for power_up in power_up_entities:
        if (player.position - power_up.position).length() < 1:
            destroy(power_up)
            print("Power Up Collected! Speed Boost Activated!")
            player_speed_boost = 2  # Speed boost activated for a short time
            power_up_entities.remove(power_up)

# Adjust lighting and environment based on the fear meter
def adjust_lighting():
    height = player.position.y
    
    # Darker environment as the player falls
    if height < 5:
        directional_light.intensity = 0.2
    elif height < 10:
        directional_light.intensity = 0.5
    else:
        directional_light.intensity = 1  # Brighter at higher levels

    # Adjust the environment color based on the fear level
    if fear_meter > 50:
        scene.color = color.rgb(50, 50, 50)  # Grayish, dark tint
    else:
        scene.color = color.white  # Normal

# Update fear meter based on proximity to shadows
def update_fear_meter():
    global fear_meter
    for shadow in shadow_entities:
        if (shadow.position - player.position).length() < 5:
            fear_meter += 1 * time.dt  # Increase fear near shadows
        else:
            fear_meter -= 0.5 * time.dt  # Slowly decrease fear when away from shadows

    # Cap the fear meter between 0 and 100
    fear_meter = max(0, min(100, fear_meter))

    # Display the fear meter value
    fear_meter_display.text = f"Fear: {int(fear_meter)}%"

    # When the fear meter is high, distort environment or play unsettling sounds
    if fear_meter > 75:
        ambient_sound.pitch = 1 + (fear_meter / 100) * 0.5  # Distorted sound
        directional_light.intensity = 0.1  # Darker when fear is high
        # Optionally, add sound effects or change the camera for a disorienting effect

# Initialize the game world
def initialize_game():
    global current_key
    create_burj_khalifa(position=(0, floor_level, 0))
    initialize_shadows()
    spawn_power_ups(count=3)

    # Dark floor
    Entity(
        model='plane',
        scale=(50, 50),
        texture='assets/dark_floor.png',
        collider='box',
        color=color.dark_gray
    )

    # Add a key collectible
    global key_entity
    key_entity = Entity(
        model='cube',
        position=(5, floor_level + 0.5, 5),
        scale=(0.3, 0.3, 0.3),
        color=color.yellow,
        texture='assets/key_texture.png'
    )

# Player setup
player = FirstPersonController()
player.position = (0, 2, 0)
fear_meter_display = Text(position=(-0.85, 0.45), scale=2, text="Fear: 0%")

# Add lighting and sky
directional_light = DirectionalLight(parent=scene, rotation=(45, -45, 0), color=color.white)
Sky()

# Game update loop
def update():
    move_shadows()
    update_fear_meter()
    collect_power_ups()

    # Check if player collects the key
    global key_entity
    if key_entity and (player.position - key_entity.position).length() < 1:
        destroy(key_entity)
        print("Key Collected! Find the Exit!")
        key_entity = None

    # Adjust environment based on fear meter and height
    adjust_lighting()

# Start the game
initialize_game()
app.run()
from ursina import *
from ursina.prefabs.first_person_controller import FirstPersonController
import random

# Initialize the app
app = Ursina()

# Set game icon (replace with your own icon file)
window.icon = 'PNG.webp'

# Variables
floor_level = 0
fear_meter = 0
power_up_entities = []
shadow_entities = []
player_speed_boost = 1  # Normal speed
ambient_sound = Audio('assets/fear_sound.mp3', loop=True, autoplay=True)  # Add your eerie sound file here

# Create Burj Khalifa structure
def create_burj_khalifa(position=(0, 0, 0)):
    heights = [2, 1.5, 1.2, 1, 0.8, 0.5, 0.2]
    current_height = 0
    for i, height in enumerate(heights):
        Entity(
            model='cylinder',
            position=(position[0], position[1] + current_height + height / 2, position[2]),
            scale=(1 - i * 0.1, height, 1 - i * 0.1),
            color=color.gray,
            texture='white_cube',
            texture_scale=(1, height)
        )
        current_height += height

    Entity(
        model='cone',
        position=(position[0], position[1] + current_height + 0.5, position[2]),
        scale=(0.2, 1, 0.2),
        color=color.gray,
        texture='white_cube'
    )

# Initialize shadow entities
def initialize_shadows():
    global shadow_entities
    for i in range(5):
        shadow = Entity(
            model='sphere',
            position=(i * 3, floor_level + 0.5, i * 3),
            scale=0.5,
            color=color.black,
            texture='assets/shadow.png'
        )
        shadow_entities.append(shadow)

# Move shadows towards the player
def move_shadows():
    for shadow in shadow_entities:
        if (shadow.position - player.position).length() < 10:
            shadow.position += (player.position - shadow.position).normalized() * time.dt * 2

# Spawn power-ups
def spawn_power_ups(count=3):
    global power_up_entities
    power_up_entities = []
    for _ in range(count):
        power_up = Entity(
            model='sphere',
            position=(random.uniform(-10, 10), floor_level + 0.5, random.uniform(-10, 10)),
            scale=0.5,
            color=color.blue,
            texture='white_cube',
        )
        power_up_entities.append(power_up)

# Collect power-ups
def collect_power_ups():
    global player_speed_boost
    for power_up in power_up_entities:
        if (player.position - power_up.position).length() < 1:
            destroy(power_up)
            print("Power Up Collected! Speed Boost Activated!")
            player_speed_boost = 2  # Speed boost activated for a short time
            power_up_entities.remove(power_up)

# Adjust lighting and environment based on the fear meter
def adjust_lighting():
    height = player.position.y
    
    # Darker environment as the player falls
    if height < 5:
        directional_light.intensity = 0.2
    elif height < 10:
        directional_light.intensity = 0.5
    else:
        directional_light.intensity = 1  # Brighter at higher levels

    # Adjust the environment color based on the fear level
    if fear_meter > 50:
        scene.color = color.rgb(50, 50, 50)  # Grayish, dark tint
    else:
        scene.color = color.white  # Normal

# Update fear meter based on proximity to shadows
def update_fear_meter():
    global fear_meter
    for shadow in shadow_entities:
        if (shadow.position - player.position).length() < 5:
            fear_meter += 1 * time.dt  # Increase fear near shadows
        else:
            fear_meter -= 0.5 * time.dt  # Slowly decrease fear when away from shadows

    # Cap the fear meter between 0 and 100
    fear_meter = max(0, min(100, fear_meter))

    # Display the fear meter value
    fear_meter_display.text = f"Fear: {int(fear_meter)}%"

    # When the fear meter is high, distort environment or play unsettling sounds
    if fear_meter > 75:
        ambient_sound.pitch = 1 + (fear_meter / 100) * 0.5  # Distorted sound
        directional_light.intensity = 0.1  # Darker when fear is high
        # Optionally, add sound effects or change the camera for a disorienting effect

# Initialize the game world
def initialize_game():
    global current_key
    create_burj_khalifa(position=(0, floor_level, 0))
    initialize_shadows()
    spawn_power_ups(count=3)

    # Dark floor
    Entity(
        model='plane',
        scale=(50, 50),
        texture='assets/dark_floor.png',
        collider='box',
        color=color.dark_gray
    )

    # Add a key collectible
    global key_entity
    key_entity = Entity(
        model='cube',
        position=(5, floor_level + 0.5, 5),
        scale=(0.3, 0.3, 0.3),
        color=color.yellow,
        texture='assets/key_texture.png'
    )

# Player setup
player = FirstPersonController()
player.position = (0, 2, 0)
fear_meter_display = Text(position=(-0.85, 0.45), scale=2, text="Fear: 0%")

# Add lighting and sky
directional_light = DirectionalLight(parent=scene, rotation=(45, -45, 0), color=color.white)
Sky()

# Game update loop
def update():
    move_shadows()
    update_fear_meter()
    collect_power_ups()

    # Check if player collects the key
    global key_entity
    if key_entity and (player.position - key_entity.position).length() < 1:
        destroy(key_entity)
        print("Key Collected! Find the Exit!")
        key_entity = None

    # Adjust environment based on fear meter and height
    adjust_lighting()

# Start the game
initialize_game()
app.run()


# Run the application
app.run()
